In [15]:
import pandas as pd
import pickle, os, re


def collect(df_dict):
    mode_dict = {'0':'interday', '1':'intraday'}
    act_dict = {'r':'relu', 't':'tanh', 's':'sigmoid'}

    hy_params = os.listdir('logs/')
    for hy_param in hy_params:
        hypers = hy_param.split('_')
        if len(hypers) < 3:
            continue

        act_hy_params = os.listdir('logs/'+hy_param+'/')
        for act_hy_param in act_hy_params:
            if len(act_hy_param.split('.')) > 1:
                continue

            items = os.listdir('logs/' + hy_param + '/' + act_hy_param + '/')
            for item in items:
                if item.endswith('.pkl'):
                    model_name = item.split('_')[0]
                    with open('logs/' + hy_param + '/' + act_hy_param + '/' + item, 'rb') as file:
                        history, test_loss, test_metrics = pickle.load(file)
                    history_df = pd.DataFrame(history)
                    insert = [int(hypers[0]), mode_dict[hypers[1]], hypers[2], act_dict[act_hy_param],
                              history_df['IC'].values[-1], history_df['val_IC'].values[-1], test_metrics]
                    df_dict[model_name].loc[len(df_dict[model_name])] = insert
    return df_dict


In [16]:
df_dict = {}
models = ['x', 'y', 'cnn', 'lstm', 'bilstm', 'tcn']
for model in models:
    df_dict[model] = pd.DataFrame(columns=['size','mode','bar','activation','train','valid','test'])
result = collect(df_dict)
for model, df in df_dict.items():
    df = df.sort_values(by=['mode', 'size'], ascending=True)
    df.index = range(len(df))
    df_dict[model] = df

In [17]:
for model, df in df_dict.items():
    print(model)
    print(df)

x
   size      mode bar activation     train     valid      test
0     1  interday  15       relu  0.157585  0.118602  0.122097
1     1  interday  15       tanh  0.126806  0.110878  0.113502
2     2  interday  15       relu  0.153810  0.121954  0.115141
3     2  interday  15       tanh  0.152344  0.109738  0.107553
4     3  interday  15       relu  0.143074  0.111522  0.111883
5     3  interday  15       tanh  0.154202  0.111011  0.100136
6     4  interday  15       relu  0.101757  0.099952  0.103036
7     4  interday  15       tanh  0.125259  0.109379  0.107225
8     5  interday  15       relu  0.149804  0.125903  0.114034
9     5  interday  15       tanh  0.118241  0.102972  0.104816
10    6  interday  15       relu  0.134866  0.119310  0.114086
11    6  interday  15       tanh  0.122918  0.111860  0.112252
12    8  interday  15       relu  0.115946  0.107686  0.106489
13    8  interday  15       tanh  0.134878  0.117650  0.109433
14    9  interday  15       relu  0.145092  0.122026 

In [19]:
a = np.array([1,2,4, np.nan, 4])

In [24]:
judge = a>3
judge

/Users/geek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.


array([False, False,  True, False,  True])

In [28]:
b = np.array([1,1,2,1,1])

In [29]:
judge*b

array([0, 0, 2, 0, 1])